<a href="https://colab.research.google.com/github/Sid00867/CatvDog-Image-Recognition/blob/main/Cat_%26_Dog_Image_Classification_(Colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from tqdm import tqdm  # progress bar
import os
import zipfile

# 'dataset.zip' file containing your images.
if not os.path.exists('./training_set'):

    zip_path = '/content/drive/MyDrive/dataset.zip'
    if os.path.exists(zip_path):
        print("Zip file found. Extracting...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall('./') # Extract to the current directory
        print("Extraction complete.")
    else:
        print("ERROR: 'dataset.zip' not found.")
        print("Please upload your zip file containing 'training_set' and 'test_set' folders.")

train_dir = './training_set/training_set'
test_dir = './test_set/test_set'


# DEVICE CONFIGURATION (USE GPU IF AVAILABLE)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Create datasets with ImageFolder
# It automatically finds classes from folder names (e.g., 'cats', 'dogs')
try:
    train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
    test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    print("Datasets and DataLoaders created successfully.")
    print(f"Training classes: {train_dataset.class_to_idx}")

except FileNotFoundError:
    print("\nERROR: Could not find the training or test directories.")
    print(f"Looked for '{train_dir}' and '{test_dir}'.")
    print("Please ensure your zip file creates these folders when extracted.")
    # Set loaders to None so the script doesn't crash later
    train_loader = None
    test_loader = None

class CatDogCNN(nn.Module):
    def __init__(self):
        super(CatDogCNN, self).__init__()
        # Input: 3 x 128 x 128
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)      # -> 32 x 128 x 128
        self.pool1 = nn.MaxPool2d(2, 2)                              # -> 32 x 64 x 64

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)     # -> 64 x 64 x 64
        self.pool2 = nn.MaxPool2d(2, 2)                              # -> 64 x 32 x 32

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)    # -> 128 x 32 x 32
        self.pool3 = nn.MaxPool2d(2, 2)                              # -> 128 x 16 x 16

        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)   # -> 128 x 16 x 16
        self.pool4 = nn.MaxPool2d(2, 2)                              # -> 128 x 8 x 8

        self.flatten_dim = 128 * 8 * 8

        self.fc1 = nn.Linear(self.flatten_dim, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = F.relu(self.conv4(x))
        x = self.pool4(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x


if train_loader and test_loader:
    # Instantiate the model and move it to the configured device (GPU/CPU)
    model = CatDogCNN().to(device)

    criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 60

    # --- TRAINING LOOP ---
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        # Use tqdm for a progress bar
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

        for inputs, targets in loop:
            # Move data to the same device as the model
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Reshape targets to match output shape and type
            targets = targets.float().unsqueeze(1)

            # Calculate loss and perform backpropagation
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

            # Update progress bar description
            loop.set_postfix(loss=loss.item())

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs} finished, Training Loss: {epoch_loss:.4f}")

    # --- TESTING LOOP ---
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    print("\nTraining Ended. Starting Testing...")
    with torch.no_grad():
        for inputs, targets in test_loader:
            # Move data to the device
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)

            # Reshape targets
            targets = targets.float().unsqueeze(1)

            loss = criterion(outputs, targets)
            test_loss += loss.item() * inputs.size(0)

            # Calculate accuracy
            preds = (outputs >= 0.5).float()
            correct += (preds == targets).sum().item()
            total += targets.size(0)

    avg_test_loss = test_loss / total
    accuracy = correct / total
    print(f"\nTest Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.4f} ({correct}/{total})")


save_prompt = input("Do you want to save the trained model weights to 'cat_dog_model_weights.pth'? (yes/no): ")

if save_prompt.lower() in ['yes', 'y']:
    save_path = 'cat_dog_model_weights.pth'
    torch.save(model.state_dict(), save_path)
    print(f"\nModel weights successfully saved to {save_path}")
else:
    print("\nModel weights were not saved.")

Using device: cuda
Datasets and DataLoaders created successfully.
Training classes: {'cats': 0, 'dogs': 1}


Epoch 1/60: 100%|██████████| 251/251 [00:30<00:00,  8.35it/s, loss=0.749]


Epoch 1/60 finished, Training Loss: 0.6827


Epoch 2/60: 100%|██████████| 251/251 [00:28<00:00,  8.72it/s, loss=0.719]


Epoch 2/60 finished, Training Loss: 0.5993


...

Epoch 58/60 finished, Training Loss: 0.0155


Epoch 59/60: 100%|██████████| 251/251 [00:28<00:00,  8.69it/s, loss=2.39e-7]


Epoch 59/60 finished, Training Loss: 0.0300


Epoch 60/60: 100%|██████████| 251/251 [00:28<00:00,  8.72it/s, loss=0.000457]


Epoch 60/60 finished, Training Loss: 0.0074

Training Ended. Starting Testing...

Test Loss: 1.8102, Test Accuracy: 0.8329 (1685/2023)
Do you want to save the trained model weights to 'cat_dog_model_weights.pth'? (yes/no): yes

Model weights successfully saved to cat_dog_model_weights.pth
